In [1]:
from sklearn import *

import numpy as np
import pandas as pd
import sklearn as sk 
import sklearn.model_selection
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
import time

# For Gokul's machine
df = pd.read_csv("./train.csv")

#array of all the summaries

og_summaries = df['summary']
summaries = df["summary"]

#array of the overalls
overall = df["overall"]

#array of product nums
products = df["amazon-id"]

og_amazon_reviews = df['reviewText']
amazon_reviews = df['reviewText']

In [2]:
# Using ratings associated with reviews as indicators of sentiment
review_sentiments = [int(overall[i] >= 5) for i in range(len(overall))]

for i in range(len(amazon_reviews)):
    if type(amazon_reviews[i])!=str:
        amazon_reviews.pop(i)
        review_sentiments.pop(i)
print(set([type(r) for r in amazon_reviews]))

#create train test split
x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(amazon_reviews, 
                                                                       review_sentiments, 
                                                                       random_state=1)


{<class 'str'>}


In [3]:
#sentiment model based on reviews
vect = HashingVectorizer()

x_train = vect.fit_transform(x_train)
x_test = vect.transform(x_test)

# We can experiment with different numbers of max iterations
# 100 iters is default
# iters = [1000]

start = time.time()
review_clf = LogisticRegression(max_iter=500)
review_clf.fit(x_train, y_train)
end = time.time()
print("Test accuracy: {}".format(100*review_clf.score(x_test, y_test)))
print("Time taken: {:.2f} seconds".format(end-start))

/Users/gokulsrin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Test accuracy: 80.91740476704831
Time taken: 4.58 seconds


In [4]:
#summary per product
summary_sentiments = [int(overall[i] >= 5) for i in range(len(overall))]
for i in range(len(summaries)):
    if type(summaries[i])!=str:
        summaries.pop(i)
        summary_sentiments.pop(i)
print(set([type(s) for s in summaries]))

{<class 'str'>}


In [5]:
x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(summaries, 
                                                                       summary_sentiments, 
                                                                       random_state=1)
# create Summary Sentiment model

x_train = vect.fit_transform(x_train)
x_test = vect.transform(x_test)

# We can experiment with different numbers of max iterations
# 100 iters is default
# iters = [1000]

start = time.time()
summary_clf = LogisticRegression(max_iter=500)
summary_clf.fit(x_train, y_train)
end = time.time()
print("Test accuracy: {}".format(100*summary_clf.score(x_test, y_test)))
print("Time taken: {:.2f} seconds".format(end-start))

Test accuracy: 77.88219197810903
Time taken: 0.87 seconds


In [6]:
summary_data = vect.transform(df['summary'].values.astype('U'))
review_data = vect.transform(df['reviewText'].values.astype('U'))

summary_predictions = summary_clf.predict(summary_data)
review_predictions = review_clf.predict(review_data)

review_predictions, summary_predictions = list(review_predictions), list(summary_predictions)

amz_review_sentiments, amz_summary_sentiments = {}, {}

for i in range(len(products)):
    if products[i] not in amz_review_sentiments:
        amz_review_sentiments[products[i]] = [review_predictions[i]]
        amz_summary_sentiments[products[i]] = [summary_predictions[i]]
    else:
        amz_review_sentiments[products[i]].append(review_predictions[i])
        amz_summary_sentiments[products[i]].append(summary_predictions[i])
        
avg_review_sentiments = [np.average(amz_review_sentiments[prod]) for prod in amz_review_sentiments]
avg_summary_sentiments = [np.average(amz_summary_sentiments[prod]) for prod in amz_summary_sentiments]

avg_review_sentiments = [[score] for score in avg_review_sentiments]
avg_summary_sentiments = [[score] for score in avg_summary_sentiments]

IndexError: list index out of range

In [ ]:
X = np.zeros((len(avg_review_sentiments), 2))
for i in range(X.shape[0]):
    X[i] = (avg_review_sentiments[i][0], avg_summary_sentiments[i][0])


In [ ]:
x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(X, random_state=1)
x_train = vect.fit_transform(x_train)
x_test = vect.transform(x_test)
clf = LogisticRegression(max_iter=500)
clf.fit(x_train, y_train)

In [ ]:
print("Weighted F1 using review sentiment and summary sentiment: {:.2f}".format(100*f1_score(y_test, 
           clf.predict(X_test), average='weighted')))